In [ ]:
import json
json_rr_data = json.load(open('/kaggle/input/d/hamnamuslihuddeen/rhetorical-role-dataset/RR_Train.json'))

In [ ]:
# json_rr_data[0]

In [ ]:
len(json_rr_data)

In [ ]:
# print(json_rr_data[225])

Now we have loaded a labelled dataset called RR_Train.json of 247 cases. With information about a case's type (criminal etc), the case doc text, the segments in the document and their respective labels.

In [ ]:
embedding_file_creation_directory = '/kaggle/working/'

In [ ]:
!pip install transformers
!pip install sentence-transformers

In [ ]:
import json
import numpy as np
import torch
from transformers import AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer, models

# Define the path to the LegalBERT model and tokenizer
model_name = "nlpaueb/legal-bert-small-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
legal_bert_model = AutoModel.from_pretrained(model_name)

# Define a SentenceTransformer model with LegalBERT
word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
legal_bert_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])



def s_bert(flag):
    # Encode the input text using LegalBERT
    embeddings = legal_bert_model.encode(flag, convert_to_tensor=True)
    return [float(x) for x in embeddings[0].tolist()]

count=0

for i in range(247):
    print(i)
    filename = embedding_file_creation_directory + "c"+str(count)+".txt"    
    if len(json_rr_data[i]['annotations'][0]['result'])==0:
        continue
    count+=1
    with open(filename, "w") as file1:
        for j in json_rr_data[i]['annotations'][0]['result']:
            temp = j['value']['text'].replace("\n", " ")
            temp = ' '.join(temp.split())  # Remove extra spaces

            flag = [temp]
            emb = s_bert(flag)
            vec_emb = " ".join(str(float('{:.5f}'.format(n))) for n in emb)
            label = j['value']['labels'][0]
            if count==0:
                file1.write(f"{vec_emb}\t{label}")
            else:
                file1.write(f"\n{vec_emb}\t{label}")


After reviewing the generated c0.txt, we can see that each row has 768 embeddings followed with the Rhetorical Role.
c0.txt contains the embedding for each part of the first document. All of the embeddings for each part used to be same btu after initializing vectorizer before every part, the embeddings have changed.